# REINVENT4 LibInvent - NVIDIA DGX Spark
## DENV NS2B-NS3 Protease Inhibitor Generation

**Hardware**: NVIDIA DGX Spark (128GB unified memory, GB10 GPU)

**Target**: Generate ~2.5M molecules in 8-12 hours

**Scaffold**: Pyrrolidine dual aromatic

---

### Quick Start
1. 确保所有文件已复制到正确位置
2. 点击 `Run All` 即可开始
3. 训练过程中可以在TensorBoard查看实时进度

---

In [1]:
# Cell 1: 环境检查和导入
import os
import sys
import shutil
import subprocess
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

print("=" * 80)
print("REINVENT4 LibInvent - DGX Spark Setup")
print("=" * 80)
print(f"\n时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Python版本: {sys.version}")
print(f"工作目录: {os.getcwd()}")

# 检查CUDA
try:
    import torch
    print(f"\nPyTorch版本: {torch.__version__}")
    print(f"CUDA可用: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA版本: {torch.version.cuda}")
        print(f"GPU设备: {torch.cuda.get_device_name(0)}")
        print(f"GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
except Exception as e:
    print(f"\n⚠️ PyTorch检查失败: {e}")

print("\n✅ 环境检查完成")

REINVENT4 LibInvent - DGX Spark Setup

时间: 2025-11-30 17:20:03
Python版本: 3.10.19 (main, Oct 21 2025, 16:38:01) [GCC 11.2.0]
工作目录: /home/august/aidd/DENV_Drug_Discovery

PyTorch版本: 2.9.1+cu130
CUDA可用: True
CUDA版本: 13.0
GPU设备: NVIDIA GB10
GPU内存: 128.5 GB

✅ 环境检查完成


/home/august/miniconda3/envs/reinvent4/lib/python3.10/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


In [2]:
# Cell 2: 路径配置
# 根据你的实际路径修改这些变量
REINVENT_HOME = Path.home() / "aidd" / "DENV_Drug_Discovery"
EXP_NAME = "spark_run15"
EXP_DIR = REINVENT_HOME / "experiments" / "runs" / EXP_NAME

print("路径配置:")
print(f"  REINVENT根目录: {REINVENT_HOME}")
print(f"  实验目录: {EXP_DIR}")

# 创建实验目录
EXP_DIR.mkdir(parents=True, exist_ok=True)
print(f"\n✅ 实验目录已创建: {EXP_DIR}")

路径配置:
  REINVENT根目录: /home/august/aidd/DENV_Drug_Discovery
  实验目录: /home/august/aidd/DENV_Drug_Discovery/experiments/runs/spark_run15

✅ 实验目录已创建: /home/august/aidd/DENV_Drug_Discovery/experiments/runs/spark_run15


In [3]:
# Cell 3: 文件完整性检查
print("检查必需文件...\n")

required_files = {
    "Prior模型": REINVENT_HOME / "priors" / "libinvent.prior",
    "Agent模型": REINVENT_HOME / "priors" / "denv_libinvent_model_v2.model",
    "Scaffold文件": REINVENT_HOME / "data" / "pyrrolidine_dual_aryl.smi",
    "QSAR模型": REINVENT_HOME / "models" / "random_forest_champion.joblib",
    "QSAR插件": REINVENT_HOME / "reinvent_plugins" / "components" / "comp_qsar_scorer.py",
}

all_exists = True
for name, path in required_files.items():
    exists = path.exists()
    status = "✅" if exists else "❌"
    print(f"{status} {name}: {path}")
    if not exists:
        all_exists = False
        print(f"   ⚠️ 文件不存在，请手动复制！")

if all_exists:
    print("\n✅ 所有必需文件检查通过")
else:
    print("\n❌ 有文件缺失，请先复制文件再继续")
    print("\n停止执行。请复制文件后重新运行。")
    raise FileNotFoundError("缺少必需文件")

检查必需文件...

✅ Prior模型: /home/august/aidd/DENV_Drug_Discovery/priors/libinvent.prior
✅ Agent模型: /home/august/aidd/DENV_Drug_Discovery/priors/denv_libinvent_model_v2.model
✅ Scaffold文件: /home/august/aidd/DENV_Drug_Discovery/data/pyrrolidine_dual_aryl.smi
✅ QSAR模型: /home/august/aidd/DENV_Drug_Discovery/models/random_forest_champion.joblib
✅ QSAR插件: /home/august/aidd/DENV_Drug_Discovery/reinvent_plugins/components/comp_qsar_scorer.py

✅ 所有必需文件检查通过


In [4]:
# Cell 4: 生成配置文件
config_content = f"""# REINVENT4 LibInvent Configuration - NVIDIA DGX Spark Optimized
# Generated at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

run_type = "staged_learning"
device = "cuda:0"
tb_logdir = "experiments/runs/{EXP_NAME}/tensorboard"
json_out_config = "experiments/runs/{EXP_NAME}/_config.json"

[parameters]
summary_csv_prefix = "experiments/runs/{EXP_NAME}/results"
use_checkpoint = true
purge_memories = false

prior_file = "priors/libinvent.prior"
agent_file = "priors/denv_libinvent_model_v2.model"
smiles_file = "data/pyrrolidine_dual_aryl.smi"

# DGX Spark优化
batch_size = 512
unique_sequences = true
randomize_smiles = false

[learning_strategy]
type = "dap"
sigma = 128
rate = 0.00015

[diversity_filter]
type = "IdenticalMurckoScaffold"
bucket_size = 15
minscore = 0.6

[inception]
smiles_file = "data/pyrrolidine_dual_aryl.smi"
memory_size = 100
sample_size = 20

[[stage]]
max_score = 0.85
min_steps = 1000
max_steps = 5000
chkpt_file = "experiments/runs/{EXP_NAME}/checkpoint_stage1.chkpt"

[stage.scoring]
type = "arithmetic_mean"

# QSAR Activity Prediction
[[stage.scoring.component]]
[stage.scoring.component.QSARScorer]
[[stage.scoring.component.QSARScorer.endpoint]]
name = "DENV_Activity"
weight = 2.5
[stage.scoring.component.QSARScorer.endpoint.params]
model_path = "models/random_forest_champion.joblib"
[stage.scoring.component.QSARScorer.endpoint.transform]
type = "double_sigmoid"
low = 4.5
high = 9.0
coef_div = 9.0
coef_si = 8.0
coef_se = 8.0

# Stereochemistry
[[stage.scoring.component]]
[stage.scoring.component.NumAtomStereoCenters]
[[stage.scoring.component.NumAtomStereoCenters.endpoint]]
name = "Stereo_Centers"
weight = 0.6
[stage.scoring.component.NumAtomStereoCenters.endpoint.transform]
type = "reverse_sigmoid"
low = 0
high = 3
k = 1.0

# Chemical Stability Alerts
[[stage.scoring.component]]
[stage.scoring.component.CustomAlerts]
[[stage.scoring.component.CustomAlerts.endpoint]]
name = "Stability_Alerts"
weight = 1.2
[stage.scoring.component.CustomAlerts.endpoint.params]
smarts = [
    "[*;r8]", "[*;r9]", "[*;r10]",
    "[#8][#8]",
    "[#6;+]",
    "C#C",
    "[NX3][NX3]",
    "[SH]",
    "[N+](=O)[O-]",
    "S(=O)(=O)Cl",
    "[F,Cl,Br,I][C,c][F,Cl,Br,I]",
    "c1ccc2c(c1)C(=O)c1c(C2=O)cccc1",
    "C1=CC(=O)C=CC1=O",
    "[C;R1]1[C;R1][C;R1][N;R1][C;R1]1([!#1])([!#1])([!#1])",
    "[C;R1]1[C;R1]([!#1])([!#1])[C;R1]([!#1])([!#1])[N;R1][C;R1]1",
]

# Drug-likeness
[[stage.scoring.component]]
[stage.scoring.component.Qed]
[[stage.scoring.component.Qed.endpoint]]
name = "QED"
weight = 0.3

[[stage.scoring.component]]
[stage.scoring.component.SAScore]
[[stage.scoring.component.SAScore.endpoint]]
name = "SA"
weight = 0.4
[stage.scoring.component.SAScore.endpoint.transform]
type = "reverse_sigmoid"
low = 1.0
high = 5.5
k = 0.8

# Molecular Properties
[[stage.scoring.component]]
[stage.scoring.component.MolecularWeight]
[[stage.scoring.component.MolecularWeight.endpoint]]
name = "MW"
weight = 0.5
[stage.scoring.component.MolecularWeight.endpoint.transform]
type = "double_sigmoid"
low = 280.0
high = 600.0
coef_div = 600.0
coef_si = 15.0
coef_se = 15.0

[[stage.scoring.component]]
[stage.scoring.component.NumHeavyAtoms]
[[stage.scoring.component.NumHeavyAtoms.endpoint]]
name = "Heavy_Atoms"
weight = 0.4
[stage.scoring.component.NumHeavyAtoms.endpoint.transform]
type = "double_sigmoid"
low = 18
high = 45
coef_div = 45.0
coef_si = 5.0
coef_se = 5.0

[[stage.scoring.component]]
[stage.scoring.component.SlogP]
[[stage.scoring.component.SlogP.endpoint]]
name = "LogP"
weight = 0.3
[stage.scoring.component.SlogP.endpoint.transform]
type = "double_sigmoid"
low = 0.5
high = 5.0
coef_div = 5.0
coef_si = 10.0
coef_se = 10.0

[[stage.scoring.component]]
[stage.scoring.component.TPSA]
[[stage.scoring.component.TPSA.endpoint]]
name = "TPSA"
weight = 0.2
[stage.scoring.component.TPSA.endpoint.transform]
type = "double_sigmoid"
low = 30.0
high = 120.0
coef_div = 120.0
coef_si = 20.0
coef_se = 20.0

[[stage.scoring.component]]
[stage.scoring.component.HBondAcceptors]
[[stage.scoring.component.HBondAcceptors.endpoint]]
name = "HBA"
weight = 0.3
[stage.scoring.component.HBondAcceptors.endpoint.transform]
type = "reverse_sigmoid"
low = 2
high = 8
k = 0.5

[[stage.scoring.component]]
[stage.scoring.component.HBondDonors]
[[stage.scoring.component.HBondDonors.endpoint]]
name = "HBD"
weight = 0.2
[stage.scoring.component.HBondDonors.endpoint.transform]
type = "reverse_sigmoid"
low = 0
high = 4
k = 0.5

[[stage.scoring.component]]
[stage.scoring.component.NumRotBond]
[[stage.scoring.component.NumRotBond.endpoint]]
name = "Rotatable_Bonds"
weight = 0.2
[stage.scoring.component.NumRotBond.endpoint.transform]
type = "reverse_sigmoid"
low = 0
high = 9
k = 0.5

[[stage.scoring.component]]
[stage.scoring.component.NumRings]
[[stage.scoring.component.NumRings.endpoint]]
name = "Ring_Count"
weight = 0.2
[stage.scoring.component.NumRings.endpoint.transform]
type = "reverse_sigmoid"
low = 2
high = 5
k = 0.5

[[stage.scoring.component]]
[stage.scoring.component.NumAromaticRings]
[[stage.scoring.component.NumAromaticRings.endpoint]]
name = "Aromatic_Rings"
weight = 0.15
[stage.scoring.component.NumAromaticRings.endpoint.transform]
type = "reverse_sigmoid"
low = 1
high = 3
k = 0.5

# toxic alerts, originated from run15
# 1. 扩展的毒性警报
[[stage.scoring.component]]
[stage.scoring.component.CustomAlerts]
[[stage.scoring.component.CustomAlerts.endpoint]]
name = "Toxicity_Alerts"
weight = 1.5  # 高权重，因为安全性重要

[stage.scoring.component.CustomAlerts.endpoint.params]
smarts = [
    # 致突变性结构
    "N=N",  # 偶氮化合物
    "[N;R0]=[N;R0][#6]",  # 重氮化合物
    "C(=O)N(N)C(=O)",  # 肼衍生物
    "[#6][N+](=O)[O-]",  # 硝基芳烃
    
    # 反应性/毒性官能团
    "C(=O)Cl",  # 酰氯
    "OS(=O)(=O)[#6]",  # 磺酸酯
    "[#6]S(=O)(=O)F",  # 磺酰氟
    "C(=O)C(=O)",  # α-二酮
    "[#6]OO[#6]",  # 过氧化物
    
    # 潜在心脏毒性
    "[#6]c1ccc(F)cc1",  # 对氟苯基（hERG风险）
    "CN1CCN(CC1)c2",  # 哌嗪片段（某些情况）
    
    # 肝毒性相关
    "c1ccccc1[N+](=O)[O-]",  # 硝基苯
    "Nc1ccc(cc1)S(=O)(=O)N",  # 磺胺类
    
    # 基因毒性
    "[#6]=[#6]-[#6]=[O,S]",  # Michael受体
    "C=CC(=O)[#6,#7,#8]",  # α,β-不饱和羰基
    "[#7]-[#7]=C",  # 腙类
    
    # 其他高风险结构
    "[Cl,Br,I][#6][Cl,Br,I]",  # 二卤代烷
    "c1cc2OCOc2cc1",  # 亚甲二氧基（某些情况）
    "[P,S][F,Cl,Br,I]",  # 含卤素的磷/硫化合物
]

# 2. PAINS (Pan-Assay INterference compoundS) 过滤
[[stage.scoring.component]]
[stage.scoring.component.CustomAlerts]
[[stage.scoring.component.CustomAlerts.endpoint]]
name = "PAINS_Filter"
weight = 0.8

[stage.scoring.component.CustomAlerts.endpoint.params]
smarts = [
    # 常见PAINS结构
    "[#6]=[#6](C(=O)O[#6])C(=O)N",  # 烯胺类
    "O=C1C=CC(=O)C=C1",  # 醌类
    "[#6]=N-N=[#6]",  # 偶氮化合物
    "c1ccc(cc1)S(=O)(=O)O",  # 芳基磺酸
    "[OH]c1ccccc1[OH]",  # 邻苯二酚
    "N=C1C=CC(=N)C=C1",  # 醌亚胺
]

# 3. 代谢稳定性（避免快速代谢的结构）
[[stage.scoring.component]]
[stage.scoring.component.CustomAlerts]
[[stage.scoring.component.CustomAlerts.endpoint]]
name = "Metabolic_Stability"
weight = 0.6

[stage.scoring.component.CustomAlerts.endpoint.params]
smarts = [
    # 易代谢位点
    "[#6]O[#6](C)C",  # 叔丁基醚（易脱烷基）
    "c1ccccc1OC",  # 苯甲醚（易O-去甲基化）
    "[#7]C(C)(C)C",  # 叔丁胺（易N-去烷基化）
    "c1ccccc1[CH3]",  # 甲苯（易氧化）
    "[#6][CH2][OH]",  # 伯醇（易氧化）
]
"""

config_path = EXP_DIR / "config.toml"
with open(config_path, 'w') as f:
    f.write(config_content)

print(f"✅ 配置文件已生成: {config_path}")
print(f"\n配置摘要:")
print(f"  - Batch size: 512")
print(f"  - Max steps: 5000")
print(f"  - 预期分子数: ~2,560,000")
print(f"  - Learning rate: 0.00015")
print(f"  - Sigma: 128")

✅ 配置文件已生成: /home/august/aidd/DENV_Drug_Discovery/experiments/runs/spark_run15/config.toml

配置摘要:
  - Batch size: 512
  - Max steps: 5000
  - 预期分子数: ~2,560,000
  - Learning rate: 0.00015
  - Sigma: 128


In [ ]:
# Cell 5: 启动REINVENT4训练
print("=" * 80)
print("启动REINVENT4训练")
print("=" * 80)
print(f"\n开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n预计运行时间: 8-12 小时")
print(f"预计生成分子: ~2,560,000 个\n")

# 切换到REINVENT目录
os.chdir(REINVENT_HOME)
print(f"当前目录: {os.getcwd()}\n")

# 准备命令
config_rel_path = config_path.relative_to(REINVENT_HOME)
log_path = EXP_DIR / "training.log"

cmd = f"reinvent {config_rel_path}"
print(f"执行命令: {cmd}")
print(f"日志文件: {log_path}\n")
print("=" * 80)
print("训练开始...")
print("=" * 80)
print()

# 执行训练（实时输出）
with open(log_path, 'w') as log_file:
    process = subprocess.Popen(
        cmd,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    for line in process.stdout:
        print(line, end='')
        log_file.write(line)
        log_file.flush()
    
    process.wait()
    
    if process.returncode == 0:
        print("\n" + "=" * 80)
        print("✅ 训练成功完成！")
        print("=" * 80)
    else:
        print("\n" + "=" * 80)
        print(f"❌ 训练失败 (返回码: {process.returncode})")
        print("=" * 80)

print(f"\n结束时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

启动REINVENT4训练

开始时间: 2025-11-30 17:21:14

预计运行时间: 8-12 小时
预计生成分子: ~2,560,000 个

当前目录: /home/august/aidd/DENV_Drug_Discovery

执行命令: reinvent experiments/runs/spark_run15/config.toml
日志文件: /home/august/aidd/DENV_Drug_Discovery/experiments/runs/spark_run15/training.log

训练开始...

17:21:17 <INFO> Started REINVENT 4.6.27 (C) AstraZeneca 2017, 2023 on 2025-11-30
17:21:17 <INFO> Command line: /home/august/miniconda3/envs/reinvent4/bin/reinvent experiments/runs/spark_run15/config.toml
17:21:17 <INFO> Reading run configuration from /home/august/aidd/DENV_Drug_Discovery/experiments/runs/spark_run15/config.toml using format toml
17:21:17 <INFO> User august on host spark-a92c
17:21:17 <INFO> Python version 3.10.19
17:21:17 <INFO> PyTorch version 2.9.1+cu130, git 5811a8d7da873dd699ff6687092c225caffcf1bb
17:21:17 <INFO> PyTorch compiled with CUDA version 13.0
17:21:17 <INFO> RDKit version 2025.09.1
17:21:17 <INFO> Platform Linux-6.14.0-1013-nvidia-aarch64-with-glibc2.39
17:21:17 <INFO> CUDA driver ve

In [6]:
# Cell 6: 结果分析 - 加载数据
print("=" * 80)
print("结果分析")
print("=" * 80)

# 查找结果CSV文件
csv_files = list(EXP_DIR.glob("results_*.csv"))
if not csv_files:
    print("\n❌ 未找到结果CSV文件")
else:
    csv_file = csv_files[0]
    print(f"\n加载结果文件: {csv_file.name}")
    
    df = pd.read_csv(csv_file)
    print(f"\n数据维度: {df.shape}")
    print(f"总生成分子数: {len(df):,}")
    
    # 基本统计
    print("\n" + "=" * 80)
    print("基本统计")
    print("=" * 80)
    
    valid = df[df['SMILES_state'] == 1]
    invalid = df[df['SMILES_state'] == 0]
    duplicates = df[df['SMILES_state'] == 2]
    
    print(f"\nValid SMILES: {len(valid):,} ({len(valid)/len(df)*100:.2f}%)")
    print(f"Invalid SMILES: {len(invalid):,} ({len(invalid)/len(df)*100:.2f}%)")
    print(f"Batch duplicates: {len(duplicates):,} ({len(duplicates)/len(df)*100:.2f}%)")
    
    # Unique分子
    unique_smiles = df['SMILES'].nunique()
    print(f"\nUnique SMILES: {unique_smiles:,} ({unique_smiles/len(df)*100:.2f}%)")
    
    # 分数统计
    print("\n" + "=" * 80)
    print("分数统计")
    print("=" * 80)
    
    if 'total_score' in df.columns:
        print(f"\nTotal Score:")
        print(f"  Mean: {df['total_score'].mean():.4f}")
        print(f"  Std: {df['total_score'].std():.4f}")
        print(f"  Min: {df['total_score'].min():.4f}")
        print(f"  Max: {df['total_score'].max():.4f}")
    
    if 'DENV_Activity' in df.columns:
        print(f"\nDENV Activity (pIC50):")
        print(f"  Mean: {df['DENV_Activity'].mean():.4f}")
        print(f"  Std: {df['DENV_Activity'].std():.4f}")
        print(f"  Min: {df['DENV_Activity'].min():.4f}")
        print(f"  Max: {df['DENV_Activity'].max():.4f}")

结果分析

加载结果文件: results_1.csv


EmptyDataError: No columns to parse from file

In [ ]:
# Cell 7: 金标准候选分子提取
print("=" * 80)
print("金标准候选分子提取")
print("=" * 80)

# 定义金标准阈值
thresholds = {
    'pIC50': 7.5,        # IC50 < ~32 nM
    'QED': 0.6,
    'SA': 5.0,
    'total_score': 0.7,
}

print(f"\n金标准阈值:")
for k, v in thresholds.items():
    print(f"  {k}: {v}")

# 筛选金标准分子
gold_mask = (
    (df['SMILES_state'] == 1) &
    (df['DENV_Activity'] >= thresholds['pIC50']) &
    (df['QED'] >= thresholds['QED']) &
    (df['SA'] <= thresholds['SA']) &
    (df['total_score'] >= thresholds['total_score'])
)

gold_df = df[gold_mask].copy()
print(f"\n金标准候选数: {len(gold_df):,}")

# 去重
gold_df_unique = gold_df.drop_duplicates(subset=['SMILES'])
print(f"去重后: {len(gold_df_unique):,}")

# 保存金标准分子
if len(gold_df_unique) > 0:
    gold_output = EXP_DIR / "gold_standard_candidates.csv"
    gold_df_unique.to_csv(gold_output, index=False)
    print(f"\n✅ 金标准候选已保存: {gold_output.name}")
    
    # 显示前10个
    print("\nTop 10金标准候选 (按pIC50排序):")
    top10 = gold_df_unique.nlargest(10, 'DENV_Activity')
    print(top10[['SMILES', 'DENV_Activity', 'QED', 'SA', 'total_score']].to_string())
else:
    print("\n⚠️ 未找到符合金标准的分子")

In [ ]:
# Cell 8: 可视化分析
print("=" * 80)
print("可视化分析")
print("=" * 80)

# 设置绘图样式
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('REINVENT4 Generation Results - DGX Spark', fontsize=16, fontweight='bold')

# 1. Total Score分布
axes[0, 0].hist(df['total_score'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['total_score'].mean(), color='red', linestyle='--', label=f'Mean: {df["total_score"].mean():.3f}')
axes[0, 0].set_xlabel('Total Score')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Total Score Distribution')
axes[0, 0].legend()

# 2. DENV Activity (pIC50)分布
axes[0, 1].hist(df['DENV_Activity'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0, 1].axvline(7.5, color='red', linestyle='--', label='Gold threshold (7.5)')
axes[0, 1].set_xlabel('Predicted pIC50')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('DENV Activity Distribution')
axes[0, 1].legend()

# 3. QED分布
axes[0, 2].hist(df['QED'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 2].axvline(df['QED'].mean(), color='red', linestyle='--', label=f'Mean: {df["QED"].mean():.3f}')
axes[0, 2].set_xlabel('QED')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('QED Distribution')
axes[0, 2].legend()

# 4. SA Score分布
axes[1, 0].hist(df['SA'], bins=50, edgecolor='black', alpha=0.7, color='purple')
axes[1, 0].axvline(df['SA'].mean(), color='red', linestyle='--', label=f'Mean: {df["SA"].mean():.3f}')
axes[1, 0].set_xlabel('SA Score')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Synthetic Accessibility Distribution')
axes[1, 0].legend()

# 5. pIC50 vs QED散点图
axes[1, 1].scatter(df['DENV_Activity'], df['QED'], alpha=0.3, s=1)
axes[1, 1].axhline(0.6, color='red', linestyle='--', alpha=0.5)
axes[1, 1].axvline(7.5, color='red', linestyle='--', alpha=0.5)
axes[1, 1].set_xlabel('Predicted pIC50')
axes[1, 1].set_ylabel('QED')
axes[1, 1].set_title('Activity vs Drug-likeness')

# 6. Step进度
step_stats = df.groupby('step')['total_score'].mean()
axes[1, 2].plot(step_stats.index, step_stats.values, linewidth=2)
axes[1, 2].set_xlabel('Training Step')
axes[1, 2].set_ylabel('Mean Total Score')
axes[1, 2].set_title('Training Progress')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plot_path = EXP_DIR / "analysis_plots.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"\n✅ 分析图表已保存: {plot_path.name}")
plt.show()

In [ ]:
# Cell 9: 生成最终报告
print("=" * 80)
print("生成最终报告")
print("=" * 80)

report = f"""
REINVENT4 LibInvent Generation Report
========================================
Experiment: {EXP_NAME}
Hardware: NVIDIA DGX Spark (128GB, GB10 GPU)
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Configuration
-------------
Batch Size: 512
Max Steps: 5000
Learning Rate: 0.00015
Sigma: 128
Diversity Filter Bucket Size: 15

Generation Statistics
--------------------
Total Molecules Generated: {len(df):,}
Valid SMILES: {len(valid):,} ({len(valid)/len(df)*100:.2f}%)
Invalid SMILES: {len(invalid):,} ({len(invalid)/len(df)*100:.2f}%)
Batch Duplicates: {len(duplicates):,} ({len(duplicates)/len(df)*100:.2f}%)
Unique Molecules: {unique_smiles:,} ({unique_smiles/len(df)*100:.2f}%)

Score Statistics
----------------
Total Score: {df['total_score'].mean():.4f} ± {df['total_score'].std():.4f}
DENV Activity (pIC50): {df['DENV_Activity'].mean():.4f} ± {df['DENV_Activity'].std():.4f}
QED: {df['QED'].mean():.4f} ± {df['QED'].std():.4f}
SA Score: {df['SA'].mean():.4f} ± {df['SA'].std():.4f}

Gold Standard Candidates
------------------------
Total Gold Candidates: {len(gold_df_unique):,}
Thresholds:
  - pIC50 ≥ 7.5 (IC50 < ~32 nM)
  - QED ≥ 0.6
  - SA ≤ 5.0
  - Total Score ≥ 0.7

Output Files
------------
1. {csv_file.name}
2. gold_standard_candidates.csv
3. analysis_plots.png
4. training.log
5. config.toml

Next Steps
----------
1. Review gold standard candidates
2. Run virtual screening with LigUnity
3. Select candidates for synthesis
4. Experimental validation

========================================
"""

report_path = EXP_DIR / "REPORT.txt"
with open(report_path, 'w') as f:
    f.write(report)

print(report)
print(f"\n✅ 报告已保存: {report_path.name}")
print("\n" + "=" * 80)
print("分析完成！")
print("=" * 80)

In [ ]:
# Cell 10: TensorBoard启动指南
print("=" * 80)
print("TensorBoard可视化")
print("=" * 80)
print("\n要启动TensorBoard实时监控，请在终端运行：\n")
print(f"cd {REINVENT_HOME}")
print(f"tensorboard --logdir experiments/runs/{EXP_NAME}/tensorboard --bind_all\n")
print("然后在浏览器打开: http://localhost:6006\n")
print("=" * 80)